In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
from multiresolution import create_figure, generate_multires_scenes_and_log_to_rerun, generate_rerun_logs_for_object_panels, log_gaussian_splatting_results_to_rerun

In [ ]:
log_gaussian_splatting_results_to_rerun()

In [ ]:

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import condorgmm
from condorgmm.utils.common import get_assets_path
import condorgmm.data as data
from condorgmm.condor.interface.camera_tracking import initialize, fast_config
from condorgmm.condor.rerun import log_state
import jax.numpy as jnp
import jax
from condorgmm.data.ycb_dataloader import YCB_MODEL_NAMES
from condorgmm.condor.interface.shared import _frame_to_intrinsics
from condorgmm.condor.interface.object_tracking import fit_object_model, default_cfg
from jax.random import key as prngkey
from condorgmm.condor.rerun import _log_gaussians
from condorgmm.condor.pose import Pose as CondorPose
import rerun as rr
import trimesh
import glob
import pandas as pd
import condorgmm.eval.fp_loader
import pickle


def gaussian_count_str(n):
    if n < 1000:
        return f"{n} Gaussians"
    return f"{n//1000}K Gaussians"


condorgmm_results_directory = (
    condorgmm.get_root_path()
    / "assets/condorgmm_bucket/runtime_accuracy_3_2__2025-03-20-19-55-42"
)

splatting_results_directory = (
    condorgmm.get_root_path()
    / "assets/condorgmm_bucket/runtime_accuracy_gsplat_3_2__2025-03-20-19-53-16"
)


In [ ]:
gspal_multires_visuals_directory = (
    condorgmm.get_root_path()
    / "assets/condorgmm_bucket/gsplat_multires_visuals__2025-04-08-18-28-53"
)
results_files = glob.glob(str(gspal_multires_visuals_directory) + "/*.pkl")
data = pickle.load(open(results_files[0], "rb"))
num_gaussians_list = list(data.keys())
num_gaussians_list.sort()
print(data[num_gaussians_list[0]].keys())

mx = jnp.max(jnp.array(data[num_gaussians_list[2]]['log_opacities']))
jnp.sum(jnp.array(data[num_gaussians_list[2]]['log_opacities']) > mx - 1.5)


In [ ]:
from condorgmm.condor.types import Gaussian
from condorgmm.condor.geometry import isovars_and_quaternion_to_cov

In [ ]:
def splat_dict_to_gaussian(d):
    n_gaussians = d['means'].shape[0]
    log_scales = jnp.array(d['log_scales'])

    is_valid = jax.vmap(
        lambda x: jnp.logical_not(jnp.any(jnp.isnan(x)))
    )(log_scales)
    mx = jnp.max(jnp.array(d['log_opacities']))
    is_valid = jnp.logical_and(
        is_valid,
        jnp.array(d['log_opacities']) > mx - 1.5
    )

    log_scales = jnp.nan_to_num(log_scales, nan=-10.)
    xyz_cov = jax.vmap(isovars_and_quaternion_to_cov)(
        jnp.exp(log_scales),
        jnp.array(d['quats'])
    )
    return Gaussian(
        idx=jnp.arange(n_gaussians),
        xyz=jnp.array(d['means']),
        xyz_cov=xyz_cov,
        rgb=jnp.array(d['rgb']),
        rgb_vars=jnp.ones_like(d['rgb']),
        mixture_weight=jnp.ones(n_gaussians),
        origin=jnp.ones(n_gaussians),
        object_idx=jnp.zeros(n_gaussians, dtype=int),
        n_frames_since_last_had_assoc=jnp.zeros(n_gaussians),
    )[is_valid]

splat_dict_to_gaussian(data[num_gaussians_list[2]])

In [ ]:
def get_plot_data():
    all_results_dfs = []
    method_names = []

    results_files = glob.glob(str(condorgmm_results_directory) + "/*.pkl")
    results_dfs = [pd.read_pickle(f) for f in results_files]
    assert len(results_dfs) == 1
    results_df = results_dfs[0]
    all_results_dfs.append(results_df)
    method_names.append("condorgmm")

    assert splatting_results_directory is not None
    splatting_results_files = glob.glob(str(splatting_results_directory) + "/*.pkl")
    splatting_results_dfs = [pd.read_pickle(f) for f in splatting_results_files]
    assert len(splatting_results_dfs) == 1
    splatting_results_df = splatting_results_dfs[0]
    all_results_dfs.append(splatting_results_df)
    method_names.append("GSplat")

    values = {}
    min_n_gaussians = 10000000000
    max_n_gaussians = 0

    for results_df, method_name in zip(all_results_dfs, method_names):
        aggregated_df = results_df.groupby(["metric", "num_gaussians", "fps"])[
                "value"
            ].apply(lambda x: x.mean())

        num_gaussians_list = []
        runtime_list = []
        add_list = []

        for index, value in aggregated_df["ADD"].items():
            num_gaussians, fps = index
            num_gaussians_list.append(num_gaussians)
            runtime_list.append(fps)
            add_list.append(value * 100.0)
            min_n_gaussians = min(min_n_gaussians, num_gaussians)
            max_n_gaussians = max(max_n_gaussians, num_gaussians)

        values[method_name] = {
            "num_gaussians_list": num_gaussians_list,
            "runtime_list": runtime_list,
            "add_list": add_list,
        }

    # Get FP ADD
    fp_ycbv_result = condorgmm.eval.fp_loader.YCBVTrackingResultLoader(
        frame_rate=1, split="train_real"
    )
    df = fp_ycbv_result.get_dataframe(all_results_dfs[0]["scene"].iloc[0])
    df = df[df["timestep"] < 100]
    df = df[df["object"] == results_df["object"].iloc[0]]
    aggregated_df = df.groupby(["metric"])["value"].apply(lambda x: x.mean())
    fp_add = aggregated_df["ADD"] * 100.0
    num_gaussians_list = [min_n_gaussians, max_n_gaussians]
    values["FP"] = {
        "add_list": [fp_add for _ in range(len(num_gaussians_list))],
        "runtime_list": [30.0 for _ in range(len(num_gaussians_list))],
    }

    return values

get_plot_data()


In [ ]:
data = get_plot_data()
line_plot_data = {
    "plot1": {
        "condorgmm": (data["num_gaussians_list"], data["per_method_values"]["condorgmm"]["runtime_list"]),
        "Gaussian Splatting": (data["num_gaussians_list"], data["per_method_values"]["GSplat"]["runtime_list"]),
        "FoundationPose": (data["num_gaussians_list"], data["per_method_values"]["FP"]["runtime_list"]),
    },
    "plot2": {
        "condorgmm": (data["num_gaussians_list"], data["per_method_values"]["condorgmm"]["add_list"]),
        "Gaussian Splatting": (data["num_gaussians_list"], data["per_method_values"]["GSplat"]["add_list"]),
        "FoundationPose": (data["num_gaussians_list"], data["per_method_values"]["FP"]["add_list"])
    },
}
line_plot_data

In [ ]:
generate_multires_scenes_and_log_to_rerun()

In [ ]:
generate_rerun_logs_for_object_panels()

## Make the figure

In [ ]:
import matplotlib.pyplot as plt
import condorgmm

# load gen3_figures/PNGs/gaussians-1k.png into a numpy array
prefix = condorgmm.get_root_path() / "notebooks/condorgmm_figures/PNGs/"
def imread(path):
    return plt.imread(prefix / path)

top_left_images = [
    [imread("gaussians-100.png"), imread("gaussians-1k.png"), imread("gaussians-10k.png")],
    [imread("reconstruction-100.png"), imread("reconstruction-1k.png"), imread("reconstruction-10k.png")],
]

In [ ]:
bottom_left_images = [
    [imread("banana-20.png"), imread("banana-200.png"), imread("banana-2k.png")],
    [imread("crackerbox-20.png"), imread("crackerbox-200.png"), imread("crackerbox-2k.png")],
]

In [ ]:
# Create dummy image data: three 2x3 arrays.
dummy_top_right = [[np.random.rand(10, 10) for j in range(3)] for i in range(2)]

# Create dummy line plot data.
x = np.linspace(0, 10, 100)
dummy_line_plot_data = {
    "plot1": {
        "condorgmm": (x, np.sin(x)),
        "Gaussian Splatting": (x, np.sin(x + 0.5)),
        "FoundationPose": (x, np.sin(x + 1.0)),
    },
    "plot2": {
        "condorgmm": (x, np.cos(x)),
        "Gaussian Splatting": (x, np.cos(x + 0.5)),
        "FoundationPose": (x, np.cos(x + 1.0)),
    },
}

cnts = [100, 1000, 10000]

fig = create_figure(
    top_left_images, dummy_top_right, cnts, bottom_left_images, [20, 200, 2000], dummy_line_plot_data
)
plt.show()
